# Lists changes in issues linked to specified epics

Specify epics

In [ ]:
epic = ['VXT-3080', 'VXT-3952']

In [ ]:
import pandas as pd
from jira import JIRA

#store credentials in ~/.rcnet file
jira = JIRA('https://kainos-evolve.atlassian.net')
jql = '"Epic Link" in (' + ", ".join(epic) + ')'

print(jql)
epicIssues = jira.search_issues(jql)

In [26]:
from pandas.tseries.offsets import BDay

changes = pd.DataFrame()
changes['epic'] = ''
changes['key'] = ''
changes['summary'] = ''
changes['updateDate'] = ''
changes['field'] = ''
changes['oldValue'] = ''
changes['newValue'] = ''
changes['author'] = ''

for issue in epicIssues:
    issue = jira.issue(issue.key, expand='changelog')
    changelog = issue.changelog
    for history in changelog.histories:
        for item in history.items:
            try:
                author = history.author.name
            except AttributeError:
                author = 'NotFound'
                
            changes = changes.append(
                {'key': issue.key,
                 'updateDate': history.created,
                 'author': author,
                 'field': item.field,
                 'oldValue': item.fromString,
                 'newValue': item.toString,
                 'summary': issue.fields.summary,
                 'epic': issue.fields.customfield_10008
                 }, ignore_index=True)


#sort values
changes.sort_values(["key", "updateDate",], inplace=True)

#show only items from last 5 business days
changes = changes.loc[(pd.to_datetime(changes['updateDate']) > (pd.to_datetime('today') - BDay(5)))]

#don't show changes in following fields
changes = changes.loc[~(changes['field'].isin(['status', 'assignee', 'Sprint', 'reporter']))]
changes

,epic,key,summary,updateDate,field,oldValue,newValue,author
703,VXT-3952,VXT-2377,PSQLException: ERROR: index row size 3056 exce...,2018-02-09T11:28:15.552+0000,Clinical Risk,Low,Medium,S.Mackin
701,VXT-3952,VXT-2377,PSQLException: ERROR: index row size 3056 exce...,2018-02-09T11:28:39.657+0000,Velocity Team,BPM Team,Gdansk Team 1,S.Mackin
699,VXT-3952,VXT-2377,PSQLException: ERROR: index row size 3056 exce...,2018-02-09T11:28:51.322+0000,priority,SHOULD,MUST,S.Mackin
698,VXT-3952,VXT-2377,PSQLException: ERROR: index row size 3056 exce...,2018-02-09T11:29:23.181+0000,Epic Link,None,VXT-3952,S.Mackin
696,VXT-3952,VXT-2377,PSQLException: ERROR: index row size 3056 exce...,2018-02-09T11:29:23.420+0000,Epic Link,None,VXT-3952,S.Mackin
697,VXT-3952,VXT-2377,PSQLException: ERROR: index row size 3056 exce...,2018-02-09T11:29:23.420+0000,labels,ITH,Frimley,S.Mackin
695,VXT-3952,VXT-2377,PSQLException: ERROR: index row size 3056 exce...,2018-02-12T09:25:42.536+0000,Fix Version,None,Frimley MVP,S.Mackin
250,VXT-3080,VXT-3160,Documentation for configuring new service task,2018-02-09T14:20:40.289+0000,In Release Plan,None,Confirmed,M.O'Reilly
248,VXT-3080,VXT-3160,Documentation for configuring new service task,2018-02-13T12:15:10.044+0000,resolution,None,Done,S.Mackin
200,VXT-3952,VXT-3587,Task listing endpoints expose sensitive clinic...,2018-02-12T09:57:39.942+0000,Epic Link,None,VXT-3952,h.rainey
